In [1]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
#df = sqlContext.read.json("s3n://patricks3db/reviews_Movies_and_TV_small.json")
df = sqlContext.read.json("s3n://patricks3db/reviews_Movies_and_TV.json")
df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [28]:
#df.take(5)

[Row(asin=u'0790734680', helpful=[0, 0], overall=5.0, reviewText=u'Incredibly well done film. The dynamics that occur between Spacey and Cusak imbue the production with a feeling of reality and believability.  Scene selection and framing are excellent. Overall very entertaining.', reviewTime=u'07 3, 2014', reviewerID=u'A10VV7RHT2VVLO', reviewerName=u'George', summary=u'Well worth the time invested.', unixReviewTime=1404345600),
 Row(asin=u'0790734680', helpful=[0, 3], overall=5.0, reviewText=u"The book is just as quirky as Savannah or is the Savannah as quirky as the book...an all time favorite that should be on every one's shelf...you never know when you might new it !!", reviewTime=u'05 13, 2007', reviewerID=u'AI46WC46W4HSA', reviewerName=u'George L. Carter "reader"', summary=u'one of the best movies of all time', unixReviewTime=1179014400),
 Row(asin=u'0790734680', helpful=[0, 0], overall=4.0, reviewText=u'Kevin Spacey always gives a credible performance.  Curious to knowhow close s

In [2]:
ratings = df
ratings = ratings.drop("helpful")
ratings = ratings.drop("reviewText")
ratings = ratings.drop("reviewTime")
ratings = ratings.drop("reviewerName")
ratings = ratings.drop("summary")
ratings = ratings.drop("unixReviewTime")
ratings.columns

['asin', 'overall', 'reviewerID']

In [3]:
ratings.persist(StorageLevel.MEMORY_AND_DISK_SER)

DataFrame[asin: string, overall: double, reviewerID: string]

In [4]:
newUserDf = sqlContext.read.json("s3n://patricks3db/mapuidreviewerid/uidreviwerid.json")

In [5]:
newUserDf.take(1)

[Row(reviewerid=u'A1CULZCDCJJGYC', uid=70850)]

In [6]:
newMovieDf = sqlContext.read.json("s3n://patricks3db/mapmidasin/midasin.json")

In [7]:
newMovieDf.take(1)

[Row(asin=u'B000IU36LM', mid=6824)]

In [8]:
newMovieDf.persist(StorageLevel.MEMORY_AND_DISK_SER)
newUserDf.persist(StorageLevel.MEMORY_AND_DISK_SER)

DataFrame[reviewerid: string, uid: bigint]

In [9]:
rawRatings1 = ratings.join(newUserDf, ratings.reviewerID == newUserDf.reviewerid)
rawRatings1.printSchema()


root
 |-- asin: string (nullable = true)
 |-- overall: double (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerid: string (nullable = true)
 |-- uid: long (nullable = true)



In [10]:
rawRatings2 = rawRatings1.join(newMovieDf, ratings.asin == newMovieDf.asin)
rawRatings2.printSchema()

root
 |-- asin: string (nullable = true)
 |-- overall: double (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerid: string (nullable = true)
 |-- uid: long (nullable = true)
 |-- asin: string (nullable = true)
 |-- mid: long (nullable = true)



In [11]:
#print rawRatings2.take(1)[0].asin

In [12]:
rawRatings3 = rawRatings2.map(lambda row: (row.uid, row.mid, row.overall)).persist(StorageLevel.MEMORY_AND_DISK_SER)

In [13]:
#rawRatings3.take(10)

In [29]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
rank = 100
numIterations = 20
model = ALS.train(rawRatings3, rank, numIterations)

In [30]:
model.recommendProducts(100000,5)

[Rating(user=100000, product=145872, rating=4.997731281397979),
 Rating(user=100000, product=116359, rating=4.475223349854297),
 Rating(user=100000, product=699, rating=4.12722687703221),
 Rating(user=100000, product=152452, rating=3.9732349022988265),
 Rating(user=100000, product=14366, rating=3.893993585838197)]

In [31]:
#type(model)

In [32]:
#type(ALS)

In [33]:
#userMoviePair = usersDf.join(moviesDf).drop('reviewerID').drop('asin').map(lambda row: (row.userID, row.movieID))

In [34]:
#userMoviePair.take(5)
#type(userMoviePair)

In [35]:
#recommendations = model.predictAll(userMoviePair)

In [36]:
#type(recommendations)

In [37]:
#recommendations.take(5)

In [39]:
path = "s3n://patricks3db/modelsComplete100r20i"
model.save(sc, path)

In [40]:
sameModel = MatrixFactorizationModel.load(sc, path)

In [41]:
sameModel.recommendProducts(1,5)

[Rating(user=1, product=163372, rating=2.4197634397691887),
 Rating(user=1, product=11214, rating=2.4013448228019882),
 Rating(user=1, product=21794, rating=2.3634405064156776),
 Rating(user=1, product=162648, rating=2.3352358957272),
 Rating(user=1, product=171413, rating=2.3346140399527098)]

In [42]:
sameModel.userFeatures().persist(StorageLevel.MEMORY_AND_DISK_SER)

PythonRDD[1093] at RDD at PythonRDD.scala:43

In [43]:
sameModel.recommendProducts(100000,5)

[Rating(user=100000, product=145872, rating=4.997731281397979),
 Rating(user=100000, product=116359, rating=4.475223349854297),
 Rating(user=100000, product=699, rating=4.12722687703221),
 Rating(user=100000, product=152452, rating=3.9732349022988265),
 Rating(user=100000, product=14366, rating=3.893993585838197)]

In [44]:
sameModel.recommendProducts(500000,5)

[Rating(user=500000, product=39645, rating=5.756030825014899),
 Rating(user=500000, product=75124, rating=5.7084549182320785),
 Rating(user=500000, product=116060, rating=5.7060261714005005),
 Rating(user=500000, product=40802, rating=5.670629651357171),
 Rating(user=500000, product=163372, rating=5.662105119226227)]

In [45]:
sameModel.recommendProducts(1000000,5)

[Rating(user=1000000, product=41512, rating=4.997829492205691),
 Rating(user=1000000, product=181607, rating=4.856613262382808),
 Rating(user=1000000, product=21794, rating=4.694383995680129),
 Rating(user=1000000, product=171413, rating=4.6922936306380425),
 Rating(user=1000000, product=174313, rating=4.664253250713721)]